In [1]:
import pandas as pd

In [2]:
df_x = pd.read_csv('processed data/X.csv')

# using bert as an sample test

In [3]:
df_50000 = df_x.sample(50000)

In [4]:
df_50000.to_csv('processed data/X_50000.csv', index=False)

In [5]:
import torch
from transformers import pipeline

# ** Twitter-RoBERTa **
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", top_k=None)

def map_roberta_prediction(text):
    scores = classifier(text)[0]  
    scores_dict = {score["label"]: score["score"] for score in scores}

    max_label = max(scores_dict, key=scores_dict.get)

    if max_label == "LABEL_1": 
        if scores_dict["LABEL_2"] > scores_dict["LABEL_0"]:
            return "positive" 
        else:
            return "negative" 
    else:
        return "positive" if max_label == "LABEL_2" else "negative"

df_50000["predict_sentiment"] = df_50000["review"].apply(map_roberta_prediction)

print(df_50000["predict_sentiment"].value_counts())

from sklearn.metrics import classification_report
if "sentiment" in df_50000.columns:
    print("Twitter-RoBERTa performance on X:")
    print(classification_report(df_50000["sentiment"], df_50000["predict_sentiment"]))


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


predict_sentiment
positive    27248
negative    22752
Name: count, dtype: int64
Twitter-RoBERTa performance on X:
              precision    recall  f1-score   support

    negative       0.79      0.73      0.76     24873
    positive       0.75      0.81      0.78     25127

    accuracy                           0.77     50000
   macro avg       0.77      0.77      0.77     50000
weighted avg       0.77      0.77      0.77     50000

